In [ ]:
import seaborn
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
seaborn.set_style("darkgrid")

In [ ]:
df = pd.read_csv("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/csmooth/results/fsl_stats_task-lefthand.csv", index_col=0)
# df = df.set_index([ "smoothing_method", "region", "fwhm", "subject", "run"])

In [ ]:
df_indexed = df.set_index(["region", "smoothing_method", "fwhm", "subject", "run"])
wmgm_ratio_df = df_indexed.xs("gm")/df_indexed.xs("wm")
wmgm_ratio_df.reset_index(inplace=True)
wmgm_ratio_df["region"] = "gm_wm"

postcentral_precentral_ratio_df = df_indexed.xs("rh_postcentral")/df_indexed.xs("rh_precentral")
postcentral_precentral_ratio_df.reset_index(inplace=True)
postcentral_precentral_ratio_df["region"] = "rh_postcentral_rh_precentral"

# postcentral_background_ratio_df = df_indexed.xs("rh_postcentral")/df_indexed.xs("background")
# postcentral_background_ratio_df.reset_index(inplace=True)
# postcentral_background_ratio_df["region"] = "rh_postcentral_background"

df = pd.concat([df, wmgm_ratio_df, 
                postcentral_precentral_ratio_df, 
                # postcentral_background_ratio_df
                ], axis=0)

In [ ]:
stats = list()
for region in np.unique(df["region"]):
    print(region)
    tmp_df = df[df["region"] == region]
    tmp_df.loc[:, "n_active"] = (tmp_df["n_active"] - tmp_df["n_active"].mean())/ tmp_df["n_active"].std()
    model = smf.mixedlm(f"n_active ~ fwhm + C(smoothing_method, Treatment(reference='Gaussian')):fwhm", 
                        tmp_df, 
                        groups="subject",
                        re_formula="1",
                        vc_formula={"run": "0 + C(run)"})
    result = model.fit()
    stats.append([region, result.fe_params.loc["fwhm"], 
                  result.pvalues.loc["fwhm"],
                  result.conf_int().loc["fwhm", 0],
                  result.conf_int().loc["fwhm", 1],
                  result.fe_params.values[2], 
                  result.pvalues.values[2],
                  result.conf_int().values[2, 0],
                  result.conf_int().values[2, 1]
                  ])
    print(result.summary())

In [ ]:
stats_df = pd.DataFrame(stats, 
             columns=["region", "Gaussian_slope", "Gaussian_pval", "Gaussian_ci_low", "Gaussian_ci_high",
                      "constrained_interaction_slope", "constrained_pval", "constrained_ci_low", "constrained_ci_high"])
stats_df["Constrained_slope"] = stats_df["Gaussian_slope"] + stats_df["constrained_interaction_slope"]
stats_df["Constrained_ci_low"] = stats_df["Gaussian_ci_low"] + stats_df["constrained_ci_low"]
stats_df["Constrained_ci_high"] = stats_df["Gaussian_ci_high"] + stats_df["constrained_ci_high"]
# reorder the rows
stats_df = stats_df.set_index("region")
stats_df = stats_df.loc[["gm_wm",
                         # "rh_postcentral_background",
                         "rh_postcentral_rh_precentral",
                         "background",
                         "wm",
                         "gm",
                         "rh_precentral",
                         "rh_postcentral",]]
stats_df.reset_index(inplace=True)
# stats_df.to_csv("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/smoothing_hlm_stats.csv", index=False)

In [ ]:
stats_df

In [ ]:
seaborn.set_palette("muted")
seaborn.set_style("whitegrid")
stats_df["region_numeric"] = np.arange(len(stats_df))
offset = 0.15
# plot guassian vs constrained slopes using a forest plot
fig, ax = plt.subplots()
ax.errorbar(stats_df["Gaussian_slope"], 
            stats_df["region_numeric"] + offset, 
            label="Gaussian", 
            xerr=[stats_df["Gaussian_slope"] - stats_df["Gaussian_ci_low"], 
                  stats_df["Gaussian_ci_high"] - stats_df["Gaussian_slope"]],
            fmt="o", capsize=0, elinewidth=2,
            color="C0")
ax.errorbar(stats_df["Constrained_slope"],
            stats_df["region_numeric"] - offset, 
            label="Constrained", 
            xerr=[stats_df["Constrained_slope"] - stats_df["Constrained_ci_low"], 
                  stats_df["Constrained_ci_high"] - stats_df["Constrained_slope"]],
            fmt="d", capsize=0, elinewidth=2,
            color="C2")
# ax.scatter(stats_df["Constrained_slope"], stats_df["region"], label="Constrained", marker="o")

xlim_max = np.max(np.abs(ax.get_xlim()))
ax.set_xlim(-xlim_max, xlim_max)
ylim = ax.get_ylim()
ax.vlines(0, *ylim, color="k", linestyle="-", alpha=0.5)
ax.set_ylim(ylim[0], ylim[1])
ax.set_yticks(ax.get_yticks()[1:-1],
              ["GM / WM Ratio",
               "Postcentral / Precentral Ratio", 
               "Background",
               "WM", 
               "GM", 
               "Precentral",
               "Postcentral",
])
# put the legend on the outside of the plot in the upper right corner
ax.legend(loc="upper left")
ax.set_xlabel("Estimated slope\n(normalized active voxels / FWHM)")
fig.savefig("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/csmooth/figures/sensory_smoothing_hlm_forest_plot.png", 
            dpi=300, bbox_inches="tight")
fig.savefig("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/csmooth/figures/sensory_smoothing_hlm_forest_plot.pdf", 
            bbox_inches="tight")

In [ ]:

import math
import matplotlib.pyplot as plt
import seaborn as sns

regions = df['region'].unique()
N = len(regions)

# choose a grid shape (square-ish)
ncols = int(math.ceil(math.sqrt(N)))
nrows = int(math.ceil(N / ncols))

fig, axes = plt.subplots(nrows, ncols, sharey=True,
                         figsize=(4 * ncols, 3 * nrows), squeeze=False)
axes_flat = axes.ravel()

for i, region in enumerate(regions):
    ax = axes_flat[i]
    subset = df[df['region'] == region]
    sns.histplot(data=subset, x='n_active', bins=100, kde=True, ax=ax)
    ax.set_title(str(region))
    ax.set_xlabel('Number of active voxels')
    ax.set_ylabel('Count')

# hide any unused axes
for j in range(i + 1, len(axes_flat)):
    fig.delaxes(axes_flat[j])

fig.tight_layout()
# optional: save figure to `figures/hist_by_region.png`
# fig.savefig(`figures/hist_by_region.png`, dpi=300, bbox_inches='tight')

In [ ]:
df.columns

In [ ]:
#compute mean, std, and skewness of n_active by region
stats = list()
for region in np.unique(df["region"]):
    tmp_df = df[df["region"] == region]
    mean_n_active = tmp_df["n_active"].mean()
    std_n_active = tmp_df["n_active"].std()
    skew_n_active = tmp_df["n_active"].skew()
    stats.append([region, mean_n_active, std_n_active, skew_n_active])
stats_df = pd.DataFrame(stats, columns=["region", "mean_n_active", "std_n_active", "skew_n_active"])
stats_df

In [ ]:



def test_smoothing_effect(df, dependent_var="n_active"):
    # Fit a hierarchical linear model
    # 'gm_wm_ratio' is the dependent variable
    # 'smoothing_kernel' is the fixed effect
    # 'subject' and 'run' are random effects
    model = smf.mixedlm(f"{dependent_var} ~ smoothing_parameter", 
                        df.reset_index(("smoothing_parameter", "subject", "run"), inplace=False), 
                        groups="subject",
                        re_formula="1",
                        vc_formula={"run": "0 + C(run)"})
    result = model.fit()
    
    # Print the summary of the model
    print(result.summary())
    return result

In [ ]:
# ratio_df = df.xs(("constrained", "rh_precentral"))/df.xs(("constrained", "rh_postcentral"))
# ratio_df.reset_index("smoothing_parameter", inplace=False)
# fig, ax = plt.subplots()
# seaborn.violinplot(data=ratio_df.reset_index("smoothing_parameter", inplace=False),
#                    x="smoothing_parameter",
#                    y="mean_zstat")
# ax.set_title("Constrained smoothing")

In [ ]:
# fig, ax = plt.subplots()
# seaborn.violinplot(data=ratio_df.reset_index("smoothing_parameter", inplace=False),
#                    x="smoothing_parameter",
#                    y="n_active")
# ax.set_title("Constrained smoothing")

In [ ]:
# ratio_df = df.xs(("Gaussian", "rh_precentral"))/df.xs(("Gaussian", "rh_postcentral"))
# ratio_df.reset_index("smoothing_parameter", inplace=False)
# fig, ax = plt.subplots()
# seaborn.violinplot(data=ratio_df.reset_index("smoothing_parameter", inplace=False),
#                    x="smoothing_parameter",
#                    y="mean_zstat")
# ax.set_title("Gaussian smoothing")

In [ ]:
# fig, ax = plt.subplots()
# seaborn.violinplot(data=ratio_df.reset_index("smoothing_parameter", inplace=False),
#                    x="smoothing_parameter",
#                    y="n_active")
# ax.set_title("Gaussian smoothing")

## GM / WM active voxel ratio

In [ ]:
fig, axes = plt.subplots(2, 2, sharey="row", sharex="col")
wmgm_ratio_df = df.xs(("Gaussian", "gm"))/df.xs(("Gaussian", "wm"))
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[1, 0])
# axes[0, 0].set_title("Gaussian smoothing")
axes[1, 0].set_ylabel("GM / WM active voxel ratio")
axes[1, 0].set_xlabel("FWHM (mm)")
print("Gaussian smoothing")
r1 = test_smoothing_effect(wmgm_ratio_df)
stats.append(["Gaussian", "gm/wm", r1.fe_params.loc["smoothing_parameter"], r1.pvalues.loc["smoothing_parameter"]])
_tmp_wmgm_ratio_df = wmgm_ratio_df
wmgm_ratio_df = df.xs(("constrained", "gm"))/df.xs(("constrained", "wm"))
wmgm_ratio_df = pd.concat([wmgm_ratio_df, _tmp_wmgm_ratio_df.loc[[0.0]]])
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[1, 1])
# axes[0, 1].set_title("Constrained smoothing")
axes[1, 1].set_ylabel("")
axes[1, 1].set_xlabel("tau")
print("Constrained smoothing")
r2 = test_smoothing_effect(wmgm_ratio_df)
stats.append(["Constrained", "gm/wm", r2.fe_params.loc["smoothing_parameter"], r2.pvalues.loc["smoothing_parameter"]])
import os
os.makedirs("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/figures", exist_ok=True)
# fig.savefig("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/figures/gm_wm_ratio.png", 
#             dpi=300, bbox_inches="tight")

### GM SNR

In [ ]:
# fig, axes = plt.subplots(1, 2, sharey=True)
gm_gauss_df = df.xs(("Gaussian", "gm"))
seaborn.violinplot(data=gm_gauss_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="tSNR", ax=axes[0, 0])
axes[0, 0].set_title("Gaussian smoothing")
axes[0, 0].set_ylabel("GM tSNR")
axes[0, 0].set_xlabel("")
print("Gaussian smoothing")
r1 = test_smoothing_effect(gm_gauss_df, "tSNR")
stats.append(["Gaussian", "GM_tSNR", r1.fe_params.loc["smoothing_parameter"], r1.pvalues.loc["smoothing_parameter"]])
gm_constrained_df = df.xs(("constrained", "gm"))
gm_constrained_df = pd.concat([gm_constrained_df, gm_gauss_df.loc[[0.0]]])
seaborn.violinplot(data=gm_constrained_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="tSNR", ax=axes[0, 1])
axes[0, 1].set_title("Constrained smoothing")
axes[0, 1].set_ylabel("")
axes[0, 1].set_xlabel("")
print("Constrained smoothing")
r2 = test_smoothing_effect(gm_constrained_df, "tSNR")
stats.append(["Constrained", "GM_tSNR", r2.fe_params.loc["smoothing_parameter"], r2.pvalues.loc["smoothing_parameter"]])

fig.savefig("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/figures/gmwm_gm_tSNR.png", 
            dpi=300, bbox_inches="tight")

## Postcentral / Precentral gyrus

### Postcentral / Precentral gyrus active voxel ratio

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True)
wmgm_ratio_df = df.xs(("Gaussian", "rh_postcentral"))/df.xs(("Gaussian", "rh_precentral"))
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[0])
axes[0].set_title("Gaussian smoothing")
axes[0].set_ylabel("Postcentral / Precentral active voxel ratio")
axes[0].set_xlabel("FWHM (mm)")
print("Gaussian smoothing")
r1 = test_smoothing_effect(wmgm_ratio_df, "n_active")
stats.append(["Gaussian", "postcentral/precentral", r1.fe_params.loc["smoothing_parameter"], r1.pvalues.loc["smoothing_parameter"]])
_tmp_wmgm_ratio_df = wmgm_ratio_df
wmgm_ratio_df = df.xs(("constrained", "rh_postcentral"))/df.xs(("constrained", "rh_precentral"))
wmgm_ratio_df = pd.concat([wmgm_ratio_df, _tmp_wmgm_ratio_df.loc[[0.0]]])
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[1])
axes[1].set_title("Constrained smoothing")
axes[1].set_ylabel("")
axes[1].set_xlabel("tau")
print("Constrained smoothing")
r2 = test_smoothing_effect(wmgm_ratio_df)
stats.append(["Constrained", "postcentral/precentral", r2.fe_params.loc["smoothing_parameter"], r2.pvalues.loc["smoothing_parameter"]])
fig.savefig("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/figures/postcentral_precentral_ratio.png", dpi=300, bbox_inches="tight")

### Postcentral / Precentral gyrus (zstat)

In [ ]:
# fig, axes = plt.subplots(1, 2, sharey=True)
# wmgm_ratio_df = df.xs(("Gaussian", "rh_postcentral"))/df.xs(("Gaussian", "rh_precentral"))
# seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
#                    x="smoothing_parameter",
#                    y="mean_zstat", ax=axes[0])
# axes[0].set_title("Gaussian smoothing")
# axes[0].set_ylabel("Postcentral / Precentral active voxel ratio")
# axes[0].set_xlabel("FWHM (mm)")
# print("Gaussian smoothing")
# test_smoothing_effect(wmgm_ratio_df, "mean_zstat")
# _tmp_wmgm_ratio_df = wmgm_ratio_df
# wmgm_ratio_df = df.xs(("constrained", "rh_postcentral"))/df.xs(("constrained", "rh_precentral"))
# wmgm_ratio_df = pd.concat([wmgm_ratio_df, _tmp_wmgm_ratio_df.loc[[0.0]]])
# seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
#                    x="smoothing_parameter",
#                    y="mean_zstat", ax=axes[1])
# axes[1].set_title("Constrained smoothing")
# axes[1].set_ylabel("")
# axes[1].set_xlabel("tau")
# print("Constrained smoothing")
# test_smoothing_effect(wmgm_ratio_df, "mean_zstat")

## Postcentral gyrus

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True)
wmgm_ratio_df = df.xs(("Gaussian", "rh_postcentral"))
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[0])
axes[0].set_title("Gaussian smoothing")
axes[0].set_ylabel("Active voxels (postcentral gyrus)")
axes[0].set_xlabel("FWHM (mm)")
print("Gaussian smoothing")
r1 = test_smoothing_effect(wmgm_ratio_df, "n_active")
stats.append(["Gaussian", "postcentral", r1.fe_params.loc["smoothing_parameter"], r1.pvalues.loc["smoothing_parameter"]])
_tmp_wmgm_ratio_df = wmgm_ratio_df
wmgm_ratio_df = df.xs(("constrained", "rh_postcentral"))
wmgm_ratio_df = pd.concat([wmgm_ratio_df, _tmp_wmgm_ratio_df.loc[[0.0]]])
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[1])
axes[1].set_title("Constrained smoothing")
axes[1].set_ylabel("")
axes[1].set_xlabel("tau")
print("Constrained smoothing")
r2 = test_smoothing_effect(wmgm_ratio_df, "n_active")
stats.append(["Constrained", "postcentral", r2.fe_params.loc["smoothing_parameter"], r2.pvalues.loc["smoothing_parameter"]])
fig.savefig("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/figures/postcentral.png", dpi=300, bbox_inches="tight")

## Precentral gyrus

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True)
wmgm_ratio_df = df.xs(("Gaussian", "rh_precentral"))
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[0])
axes[0].set_title("Gaussian smoothing")
axes[0].set_ylabel("Active voxels (precentral gyrus)")
axes[0].set_xlabel("FWHM (mm)")
print("Gaussian smoothing")
r1 = test_smoothing_effect(wmgm_ratio_df, "n_active")
stats.append(["Gaussian", "precentral", r1.fe_params.loc["smoothing_parameter"], r1.pvalues.loc["smoothing_parameter"]])
_tmp_wmgm_ratio_df = wmgm_ratio_df
wmgm_ratio_df = df.xs(("constrained", "rh_precentral"))
wmgm_ratio_df = pd.concat([wmgm_ratio_df, _tmp_wmgm_ratio_df.loc[[0.0]]])
seaborn.violinplot(data=wmgm_ratio_df.reset_index("smoothing_parameter", inplace=False),
                   x="smoothing_parameter",
                   y="n_active", ax=axes[1])
axes[1].set_title("Constrained smoothing")
axes[1].set_ylabel("")
axes[1].set_xlabel("tau")
print("Constrained smoothing")
r2 = test_smoothing_effect(wmgm_ratio_df)
stats.append(["Constrained", "precentral", r2.fe_params.loc["smoothing_parameter"], r2.pvalues.loc["smoothing_parameter"]])
fig.savefig("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/figures/precentral.png", dpi=300, bbox_inches="tight")

In [ ]:
stats_df = pd.DataFrame(stats, columns=["smoothing_method", "region", "slope", "pval"])
stats_df.to_csv("/Users/david.ellis/Box Sync/Aizenberg_Documents/Papers/Bayesian/precision/smoothing_hlm_stats.csv")
stats_df